In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install the required packages if they are not already installed.
!pip install datasets transformers


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-

In [ ]:
# --- Prerequisites ---
# 1. Ensure you are using a GPU runtime in Colab (recommended for faster detection).
# 2. Ensure your .jsonl files are in the specified folder on Google Drive.
# 3. Set the folder path and the file index range in Step 3 below.

# --- Step 0: Install necessary packages ---
!pip install langdetect transformers torch accelerate datasets sentencepiece -q
print("Required packages installed.")

# --- Step 1: Import required libraries ---
from collections import Counter
import os
import io
from google.colab import drive
import datetime
import torch
import gc
import logging
import json # For manual JSON parsing

# Added for Hugging Face Transformers & Datasets
import transformers
import datasets
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset, load_dataset

# --- Configure Logging ---
logging.getLogger("transformers").setLevel(logging.WARNING)

print("Libraries imported.")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

# --- Step 2: Mount Google Drive ---
print("\nMounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise

# --- Step 3: Configuration ---
jsonl_folder_path = '/content/drive/MyDrive/unzipped_longeval_abstracts' # <--- SET FOLDER PATH
start_file_index = 21
end_file_index = 21
filename_prefix = "documents_"
filename_suffix = ".jsonl"

# --- Parameters ---
map_batch_size = 128
lang_id_pipeline_batch_size = 128
max_length_langid = 512

# --- Step 4: Language Detection Model/Pipeline Setup (Load Once) ---
print("\n--- Setting up Language Detection Model (Loading Once) ---")
start_time_setup_langid = datetime.datetime.now()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU detected. Using device: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
    gc.collect()
else:
    device = torch.device("cpu")
    print("Warning: GPU not detected. Using CPU (might be slow).")

lang_id_model_name = "papluca/xlm-roberta-base-language-detection"
lang_id_pipeline = None
lang_id_model = None
lang_id_tokenizer = None
try:
    print(f"Loading LangID model '{lang_id_model_name}'...")
    lang_id_tokenizer = AutoTokenizer.from_pretrained(lang_id_model_name)
    lang_id_model = AutoModelForSequenceClassification.from_pretrained(lang_id_model_name).to(device)
    lang_id_pipeline = pipeline("text-classification", model=lang_id_model, tokenizer=lang_id_tokenizer, device=0 if device.type == 'cuda' else -1)
    print("LangID Pipeline loaded successfully.")
except Exception as e:
    print(f"Error loading LangID model: {e}")
    raise

end_time_setup_langid = datetime.datetime.now()
print(f"Time taken for LangID setup: {end_time_setup_langid - start_time_setup_langid}")


# --- Step 5: Loop Through Files and Process ---

print(f"\n--- Starting Processing Loop for Files {start_file_index} to {end_file_index} ---")

for i in range(start_file_index, end_file_index + 1):
    current_filename = f"{filename_prefix}{i:06d}{filename_suffix}"
    current_file_path = os.path.join(jsonl_folder_path, current_filename)

    print(f"\n===== Processing File: {current_filename} =====")

    # --- 5.1: Load Data (Manual Validation for 'abstract', skips None/non-string) ---
    print(f"\n--- Attempting Manual JSONL Validation for 'abstract' in: {current_filename} ---")

    valid_lines_data = [] # Store dicts like {'abstract': '...'}
    error_line_num = -1
    json_error_detail = None
    line_content_on_error = ""
    skipped_null_abstract_count = 0
    skipped_non_string_abstract_count = 0
    skipped_missing_abstract_count = 0
    ds = None

    try:
        if not os.path.exists(current_file_path):
            raise FileNotFoundError(f"File not found at path: {current_file_path}")

        print(f"   Opening file: {current_file_path} with encoding='utf-8'")
        with open(current_file_path, 'r', encoding='utf-8') as f:
            for line_idx, line in enumerate(f):
                line_num = line_idx + 1
                stripped_line = line.strip()
                if not stripped_line:
                    continue # Skip empty lines
                try:
                    json_obj = json.loads(stripped_line)

                    # --- Handle 'abstract' field ---
                    if 'abstract' not in json_obj:
                        # print(f"   ---> Skipping line {line_num}: Missing 'abstract' key.") # Optional verbose output
                        skipped_missing_abstract_count += 1
                        continue # Skip this line
                    elif json_obj['abstract'] is None:
                        # print(f"   ---> Skipping line {line_num}: 'abstract' is null/None.") # Optional verbose output
                        skipped_null_abstract_count += 1
                        continue # ***** CHANGE: Skip line instead of stopping *****
                    elif not isinstance(json_obj['abstract'], str):
                        # print(f"   ---> Skipping line {line_num}: 'abstract' is not a string (Type: {type(json_obj['abstract'])}).") # Optional verbose output
                        skipped_non_string_abstract_count += 1
                        continue # ***** CHANGE: Skip line instead of stopping *****
                    elif not json_obj['abstract'].strip():
                         # Also skip if abstract is an empty string or only whitespace
                         # print(f"   ---> Skipping line {line_num}: 'abstract' is empty or whitespace.") # Optional verbose output
                         skipped_null_abstract_count += 1 # Count it with nulls/empties
                         continue

                    # If abstract is a valid, non-empty string, store it
                    valid_lines_data.append({'abstract': json_obj['abstract']})

                except json.JSONDecodeError as e:
                    # Still treat JSON errors as fatal for the file
                    error_line_num = line_num
                    json_error_detail = e
                    line_content_on_error = stripped_line[:200] + "..."
                    print(f"   !!! Invalid JSON detected on line {error_line_num}: {e} !!!")
                    print(f"       Content preview: {line_content_on_error}")
                    break # Stop validation on first JSON error

    except FileNotFoundError:
        print(f"!!! File Not Found: {current_file_path}. Skipping. !!!")
        del valid_lines_data[:]
        gc.collect()
        continue
    except Exception as e:
        print(f"!!! Error reading file {current_file_path}: {e}. Skipping. !!!")
        del valid_lines_data[:]
        gc.collect()
        continue

    # --- Print skip counts ---
    print(f"   Skipped {skipped_missing_abstract_count} lines due to missing 'abstract' key.")
    print(f"   Skipped {skipped_null_abstract_count} lines due to null/empty/whitespace 'abstract'.")
    print(f"   Skipped {skipped_non_string_abstract_count} lines due to non-string 'abstract'.")

    # --- Check results of manual validation ---
    if error_line_num != -1:
        print(f"--- Manual validation failed for {current_filename} due to JSON error on line {error_line_num} ({json_error_detail}). Cannot proceed with this file. ---")
        del valid_lines_data[:]
        gc.collect()
        continue
    elif not valid_lines_data:
         # Handles case where file ONLY contained invalid abstracts or was empty after skipping
         print(f"--- No valid abstracts found to process in {current_filename} after skipping invalid lines. Skipping file processing. ---")
         gc.collect()
         continue
    else:
        # --- If validation found valid abstracts, create Dataset object ---
        print(f"--- Manual validation completed for {current_filename}. Found {len(valid_lines_data)} valid abstracts to process. ---")
        start_time_load = datetime.datetime.now()
        try:
            ds = Dataset.from_list(valid_lines_data)
            print(f"   Dataset object created successfully from validated abstracts for '{current_filename}'.")
            if 'abstract' not in ds.column_names:
                 raise ValueError("'abstract' column unexpectedly missing after Dataset.from_list")

        except Exception as e:
            print(f"!!! Error creating Dataset object from validated abstracts for '{current_filename}': {e}. Skipping. !!!")
            if 'ds' in locals() and ds is not None: del ds
            del valid_lines_data[:]
            gc.collect()
            continue

        end_time_load = datetime.datetime.now()
        del valid_lines_data[:]
        gc.collect()

        if ds is None:
            print(f"!!! Dataset object is None after creation attempt for '{current_filename}'. Skipping processing. !!!")
            continue

        print(ds)
        print(f"Time taken for creating Dataset object for '{current_filename}': {end_time_load - start_time_load}\n")

    # === If we reach here, 'ds' is valid and contains the 'abstract' column ===

    # --- 5.2: Language Detection ---
    print(f"\n--- Running Language Detection on Abstracts for {current_filename} ---")

    def detect_language_batch(batch):
        texts = batch["abstract"]
        detected_lang = [None] * len(texts)
        non_empty_texts = []
        non_empty_indices = []
        # We already skipped null/empty strings during validation, but double check here just in case
        for i_batch, text in enumerate(texts):
            if text is None or not isinstance(text, str) or text.strip() == "":
                detected_lang[i_batch] = "empty" # Should ideally not happen if validation worked
            else:
                non_empty_texts.append(text)
                non_empty_indices.append(i_batch)
        if non_empty_texts:
            try:
                predictions = lang_id_pipeline(
                    non_empty_texts,
                    batch_size=lang_id_pipeline_batch_size,
                    truncation=True, padding=True, max_length=max_length_langid
                )
                for idx, pred in zip(non_empty_indices, predictions):
                    detected_lang[idx] = pred["label"]
            except Exception as e_pipe:
                print(f"  Error in lang_id_pipeline during batch for {current_filename}: {e_pipe}")
                for idx in non_empty_indices:
                    detected_lang[idx] = "error"
        batch["detected_lang"] = detected_lang
        return batch

    start_time_map_langid = datetime.datetime.now()
    detected_ds = None
    try:
        detected_ds = ds.map(detect_language_batch, batched=True, batch_size=map_batch_size, num_proc=1)
        end_time_map_langid = datetime.datetime.now()
        print(f"Language detection for '{current_filename}' completed in {end_time_map_langid - start_time_map_langid}")
    except Exception as e:
        print(f"!!! Error during language detection map for '{current_filename}': {e}. Skipping statistics for this file. !!!")
        detected_ds = None

    # --- 5.3: Display Language Statistics ---
    print(f"\n--- Language Statistics for 'abstract' field in: {current_filename} ---")
    if detected_ds is not None and 'detected_lang' in detected_ds.column_names:
        try:
            language_counts = Counter(detected_ds["detected_lang"])
            total_items = len(detected_ds) # This count now only includes lines with valid abstracts
            print(f"Total valid abstracts processed in {current_filename}: {total_items}") # Updated label
            print("--- Language Distribution ---")
            if total_items > 0:
                for lang, count in language_counts.most_common():
                    percentage = (count / total_items) * 100
                    print(f"  {lang}: {count} ({percentage:.2f}%)")
            else:
                print("  No valid abstracts were processed.")

            # Summary Stats reflect only the processed items
            en_count = language_counts.get('en', 0)
            # 'empty' count here refers to abstracts that became empty after pipeline processing,
            # not the ones skipped earlier. Should be low/zero if validation worked.
            empty_count = language_counts.get('empty', 0)
            error_count = language_counts.get('error', 0)
            non_english_count_detected = total_items - en_count - empty_count - error_count

            print("\n--- Summary (based on processed abstracts) ---")
            print(f"English ('en'): {en_count}")
            print(f"Non-English (excluding empty/error): {non_english_count_detected}")
            print(f"Pipeline Processing Errors ('error'): {error_count}")
            # Optional: Could add the skipped counts here for full context
            # print(f"(Skipped lines: Missing={skipped_missing_abstract_count}, Null/Empty={skipped_null_abstract_count}, Non-String={skipped_non_string_abstract_count})")

        except Exception as e:
            print(f"!!! Error calculating statistics for '{current_filename}': {e} !!!")
    else:
        print(f"Could not calculate statistics for '{current_filename}' - detection may have failed or no valid abstracts were found.")

    # --- 5.4: Clean up memory for the current file ---
    print(f"--- Cleaning up memory after processing {current_filename} ---")
    if 'ds' in locals() and ds is not None: del ds
    if 'detected_ds' in locals() and detected_ds is not None: del detected_ds
    gc.collect()

# --- End of Loop ---
print("\n===== Finished Processing All Files =====")


# --- Step 6: Final Cleanup (Model) ---
print("\n--- Final Cleanup (Language Detection Model) ---")
del lang_id_pipeline
del lang_id_model
del lang_id_tokenizer
gc.collect()
if device.type == 'cuda':
    torch.cuda.empty_cache()
print("Cleanup complete.")

print("\n--- Script Finished ---")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.8 MB/s eta 0:00:00
Required packages installed.
Libraries imported.
PyTorch version: 2.6.0+cu124
T

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Device set to use cuda:0


LangID Pipeline loaded successfully.
Time taken for LangID setup: 0:00:09.520831

--- Starting Processing Loop for Files 21 to 21 ---

===== Processing File: documents_000021.jsonl =====

--- Attempting Manual JSONL Validation for 'abstract' in: documents_000021.jsonl ---
   Opening file: /content/drive/MyDrive/unzipped_longeval_abstracts/documents_000021.jsonl with encoding='utf-8'
   Skipped 0 lines due to missing 'abstract' key.
   Skipped 2617 lines due to null/empty/whitespace 'abstract'.
   Skipped 0 lines due to non-string 'abstract'.
--- Manual validation completed for documents_000021.jsonl. Found 11648 valid abstracts to process. ---
   Dataset object created successfully from validated abstracts for 'documents_000021.jsonl'.
Dataset({
    features: ['abstract'],
    num_rows: 11648
})
Time taken for creating Dataset object for 'documents_000021.jsonl': 0:00:00.132599


--- Running Language Detection on Abstracts for documents_000021.jsonl ---


Map:   0%|          | 0/11648 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Language detection for 'documents_000021.jsonl' completed in 0:02:45.989820

--- Language Statistics for 'abstract' field in: documents_000021.jsonl ---
Total valid abstracts processed in documents_000021.jsonl: 11648
--- Language Distribution ---
  en: 9607 (82.48%)
  es: 496 (4.26%)
  pt: 369 (3.17%)
  nl: 216 (1.85%)
  ru: 161 (1.38%)
  fr: 148 (1.27%)
  vi: 120 (1.03%)
  sw: 99 (0.85%)
  it: 80 (0.69%)
  ur: 58 (0.50%)
  tr: 55 (0.47%)
  ja: 42 (0.36%)
  de: 36 (0.31%)
  th: 34 (0.29%)
  el: 31 (0.27%)
  zh: 29 (0.25%)
  hi: 19 (0.16%)
  pl: 19 (0.16%)
  ar: 17 (0.15%)
  bg: 12 (0.10%)

--- Summary (based on processed abstracts) ---
English ('en'): 9607
Non-English (excluding empty/error): 2041
Pipeline Processing Errors ('error'): 0
--- Cleaning up memory after processing documents_000021.jsonl ---

===== Finished Processing All Files =====

--- Final Cleanup (Language Detection Model) ---
Cleanup complete.

--- Script Finished ---


In [ ]:
# --- Prerequisites ---
# 1. Ensure you are using a GPU runtime in Colab.
# 2. Ensure your .jsonl files are already extracted into a folder on Google Drive.
# 3. Set the folder path and specific filename in Step 3 below.

# --- Step 0: Install necessary packages ---
!pip install langdetect transformers torch accelerate datasets sentencepiece -q
print("Required packages installed.")

# --- Step 1: Import required libraries ---
import json
from collections import Counter, defaultdict
import os
import io
from google.colab import drive
# *** ADDED files FOR DOWNLOAD ***
from google.colab import files
import datetime
import torch
import gc # Garbage Collector interface
import logging # For cleaner logs

# Added for Hugging Face Transformers & Datasets
import transformers
import datasets
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import Dataset, load_dataset

# --- Configure Logging ---
logging.getLogger("transformers").setLevel(logging.WARNING)

print("Libraries imported.")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

# --- Step 2: Mount Google Drive ---
print("\nMounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully.")

# --- Step 3: Configuration ---
# !!! 1. Set this path to the FOLDER containing the extracted .jsonl files !!!
jsonl_folder_path = '/content/drive/MyDrive/unzipped_longeval_abstracts' # <--- SET FOLDER PATH

# !!! 2. Set the EXACT FILENAME of the single .jsonl file you want to process !!!
specific_filename_to_process = "documents_000021.jsonl" # <--- SET EXACT FILENAME

# --- Parameters ---
map_batch_size = 128
lang_id_pipeline_batch_size = 256
translation_pipeline_batch_size = 128 # Keep or adjust based on VRAM

# --- Step 4: Language Detection Model/Pipeline Setup ---
print("\n--- Setting up Language Detection ---")
start_time_setup_langid = datetime.datetime.now()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU detected. Using device: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Warning: GPU not detected. Using CPU.")

lang_id_model_name = "papluca/xlm-roberta-base-language-detection"
print(f"Loading LangID model '{lang_id_model_name}'...")
lang_id_pipeline = None
try:
    lang_id_tokenizer = AutoTokenizer.from_pretrained(lang_id_model_name)
    lang_id_model = AutoModelForSequenceClassification.from_pretrained(lang_id_model_name).to(device)
    lang_id_pipeline = pipeline("text-classification", model=lang_id_model, tokenizer=lang_id_tokenizer, device=0 if device.type == 'cuda' else -1)
    print("LangID Pipeline loaded successfully.")
except Exception as e:
     print(f"Error loading LangID model: {e}"); raise

end_time_setup_langid = datetime.datetime.now()
print(f"    Time taken for LangID setup: {end_time_setup_langid - start_time_setup_langid}")

# --- Step 5: Load Dataset ---
print("\n--- Loading Dataset ---")
data_files = os.path.join(jsonl_folder_path, specific_filename_to_process)

if not os.path.exists(data_files):
     raise FileNotFoundError(f"Input file not found: {data_files}")

start_time_load = datetime.datetime.now()
# Load 'id' and 'title' columns specifically
try:
    ds = load_dataset("json", data_files=data_files, split="train", features=datasets.Features({
        'id': datasets.Value('string'), # Keep ID if needed
        'title': datasets.Value('string') # <<<--- Load 'title'
        # Add other needed columns here, or comment out 'features' to load all
    }))
    print("Dataset loaded with specific features ('id', 'title').")
except Exception as e:
    print(f"Error loading dataset with specific features, trying to load all: {e}")
    ds = load_dataset("json", data_files=data_files, split="train")
    print("Dataset loaded with all features.")


end_time_load = datetime.datetime.now()
# Check if 'title' column exists after loading
if 'title' not in ds.column_names:
    raise ValueError("The loaded dataset does not contain a 'title' column. Please check the JSONL structure or feature loading.")
print(ds)
print(f"Time taken for loading: {end_time_load - start_time_load}\n")

# --- Step 6: Language Detection using .map (on Titles) ---
print("\n--- Running Language Detection on Titles ---")
# Define the detection function
def detect_language_batch(batch):
    # Check if 'title' column exists
    if "title" not in batch:
        print("Error: 'title' column not found in batch!")
        batch["detected_lang"] = ["error_missing_title"] * len(batch[next(iter(batch))])
        return batch

    texts = batch["title"] # <<<--- Use 'title' column
    detected_lang = [None] * len(texts)
    non_empty_texts = []
    non_empty_indices = []
    for i, text in enumerate(texts):
        # Titles might still be None or empty
        if text is None or text.strip() == "":
            detected_lang[i] = "empty"
        else:
            non_empty_texts.append(text)
            non_empty_indices.append(i)
    if non_empty_texts:
        try:
            # Titles are shorter, max_length 512 is likely excessive but safe
            predictions = lang_id_pipeline(
                non_empty_texts,
                batch_size=lang_id_pipeline_batch_size,
                truncation=True, padding=True, max_length=512
            )
            for idx, pred in zip(non_empty_indices, predictions):
                detected_lang[idx] = pred["label"]
        except Exception as e:
            print(f"Error in lang_id_pipeline within map: {e}")
            for idx in non_empty_indices:
                 detected_lang[idx] = "error"
    batch["detected_lang"] = detected_lang
    return batch

start_time_map_langid = datetime.datetime.now()
try:
    ds = ds.map(detect_language_batch, batched=True, batch_size=map_batch_size, num_proc=1)
    end_time_map_langid = datetime.datetime.now()
    print(f"Language detection completed in {end_time_map_langid - start_time_map_langid}")
except Exception as e:
    print(f"Error during language detection map: {e}")
    raise

# --- Step 7: Filter Non-English Titles ---
print("\n--- Filtering Non-English Titles ---")
start_time_filter = datetime.datetime.now()
exclude_langs = ['en', 'empty', 'error']
try:
    # Ensure 'detected_lang' column exists before filtering
    if 'detected_lang' not in ds.column_names:
        raise ValueError("'detected_lang' column not found after mapping.")
    non_en_ds = ds.filter(lambda example: example['detected_lang'] not in exclude_langs)
    num_non_english = len(non_en_ds)
except Exception as e:
    print(f"Error during filtering: {e}.")
    num_non_english = 0
    non_en_ds = None

end_time_filter = datetime.datetime.now()
print(f"Found {num_non_english} non-English titles to translate.")
print(f"Time taken for filtering: {end_time_filter - start_time_filter}")

# --- Step 8: Load Translation Model/Pipeline ---
translation_pipeline = None
if num_non_english > 0:
    print("\n--- Setting up Translation Model ---")
    start_time_setup_trans = datetime.datetime.now()
    trans_model_name = "facebook/m2m100_418M"
    print(f"Loading translation model '{trans_model_name}'...")
    try:
        trans_tokenizer = AutoTokenizer.from_pretrained(trans_model_name)
        trans_model = AutoModelForSeq2SeqLM.from_pretrained(trans_model_name).to(device)
        translation_pipeline = pipeline("translation",
                                        model=trans_model,
                                        tokenizer=trans_tokenizer,
                                        device=0 if device.type == 'cuda' else -1)
        print("Translation Pipeline loaded successfully.")
    except Exception as e:
        print(f"Error loading translation model: {e}")
        translation_pipeline = None
    end_time_setup_trans = datetime.datetime.now()
    print(f"    Time taken for translation setup: {end_time_setup_trans - start_time_setup_trans}")

# --- Step 9: Translate Non-English Titles & DOWNLOAD ---
translated_ds = None # Initialize translated_ds
if num_non_english > 0 and translation_pipeline and non_en_ds is not None:
    print("\n--- Running Translation on Titles ---")

    # Define the OPTIMIZED translation function for map (groups by language)
    def translate_batch(batch):
        """Translates titles in a batch to English using M2M100, grouping by source language."""
        start_time_batch_trans = datetime.datetime.now()
        texts_by_lang = defaultdict(list)
        indices_by_lang = defaultdict(list)
        original_indices_processed = {}

        # Check if 'title' and 'detected_lang' columns exist
        if "title" not in batch or "detected_lang" not in batch:
             print("Error: Missing 'title' or 'detected_lang' column in batch for translation!")
             batch["translated_title"] = ["Error: Missing input column"] * len(batch[next(iter(batch))])
             return batch

        for i, (text, lang) in enumerate(zip(batch["title"], batch["detected_lang"])): # <<<--- Use 'title' column
            if text and text.strip() and lang:
                texts_by_lang[lang].append(text)
                indices_by_lang[lang].append(i)
                original_indices_processed[i] = None

        translations = [None] * len(batch["title"]) # <<<--- Use length of 'title'
        processed_count = 0
        error_count = 0

        if texts_by_lang:
            for lang_code, texts_list in texts_by_lang.items():
                try:
                    # Titles are shorter, max_length 256 is likely okay
                    pipeline_output = translation_pipeline(
                        texts_list,
                        src_lang=lang_code,
                        tgt_lang='en',
                        batch_size=translation_pipeline_batch_size,
                        truncation=True,
                        max_length=256 # Adjust if needed
                    )
                    translated_texts = [p['translation_text'] for p in pipeline_output]
                    processed_count += len(translated_texts)
                    original_indices = indices_by_lang[lang_code]
                    for original_idx, translated_text in zip(original_indices, translated_texts):
                        translations[original_idx] = translated_text
                except Exception as e:
                    print(f"        ERROR translating group for lang '{lang_code}': {e}")
                    error_count += len(texts_list)
                    original_indices = indices_by_lang[lang_code]
                    for original_idx in original_indices:
                        translations[original_idx] = f"Error: Translation failed ({lang_code})"

        for i in range(len(translations)):
             if i not in original_indices_processed and translations[i] is None:
                 translations[i] = "Error: Skipped"

        batch["translated_title"] = translations
        end_time_batch_trans = datetime.datetime.now()
        gc.collect()
        return batch

    start_time_map_trans = datetime.datetime.now()
    print(f"Applying translation map (batch size: {map_batch_size})...")
    try:
        # Apply to the non_en_ds which contains 'title' and 'detected_lang'
        translated_ds = non_en_ds.map(translate_batch, batched=True, batch_size=map_batch_size, num_proc=1)
        end_time_map_trans = datetime.datetime.now()
        print(f"Translation completed in {end_time_map_trans - start_time_map_trans}")

        # --- *** NEW: Prepare and Download JSONL File *** ---
        if translated_ds and len(translated_ds) > 0:
            print("\nPreparing JSONL file for download...")
            output_filename = "translated_titles.jsonl"
            # Use Colab's temporary storage
            output_filepath = f"/content/{output_filename}"

            try:
                with open(output_filepath, 'w', encoding='utf-8') as f_out:
                    for record in translated_ds:
                        # Create a dictionary with desired fields
                        output_record = {
                            "id": record.get("id"),
                            "original_title": record.get("title"),
                            "detected_language": record.get("detected_lang"),
                            "translated_title": record.get("translated_title")
                        }
                        # Write the dictionary as a JSON line
                        json_line = json.dumps(output_record, ensure_ascii=False)
                        f_out.write(json_line + '\n')

                print(f"JSONL file created at {output_filepath}. Triggering download...")
                # Trigger browser download
                files.download(output_filepath)
                print("Download initiated.")

            except Exception as e:
                print(f"Error creating or downloading JSONL file: {e}")
        else:
             print("\nNo translated data to download.")
        # --- *** END of Download Section *** ---

        # Display some examples (Optional, can be commented out if download is primary goal)
        print("\n--- Translation Examples (Titles) ---")
        num_examples = 5
        for i in range(min(num_examples, len(translated_ds))):
            print(f"Original Lang: {translated_ds[i]['detected_lang']}")
            print(f"Original Title: {translated_ds[i]['title']}")
            print(f"Translated Title: {translated_ds[i]['translated_title']}")
            print("-" * 20)

    except Exception as e:
        print(f"Error during translation map: {e}")


elif num_non_english > 0:
    print("\n--- Translation Skipped (Pipeline failed to load) ---")
else:
    print("\n--- Translation Skipped (No non-English titles found) ---")


# --- Step 10: Final Language Counts (Original Detection on Titles) ---
print("\n--- Final Language Counts (from initial detection on Titles) ---")
# Check if dataset 'ds' and column 'detected_lang' exist before counting
if 'ds' in locals() and hasattr(ds, 'column_names') and 'detected_lang' in ds.column_names:
    language_counts = Counter(ds["detected_lang"])
    total_items = len(ds) # Changed variable name for clarity
    print(f"Total titles processed initially: {total_items}")
    print("--- Aggregated Language Distribution ---")
    for lang, count in language_counts.most_common():
        print(f"  {lang}: {count}")

    print(f"\n--- Final Summary (based on initial detection) ---")
    non_english_count_detected = sum(count for lang, count in language_counts.items() if lang not in ['en', 'empty', 'error'])
    print(f"Total number of non-English titles identified initially (excluding errors/empty): {non_english_count_detected}")
else:
    print("Could not calculate final counts - initial dataset 'ds' or 'detected_lang' column may be missing or failed.")


print("\n--- Script Finished ---")

Required packages installed.
Libraries imported.
PyTorch version: 2.6.0+cu124
Transformers version: 4.51.1
Datasets version: 3.5.0

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.

--- Setting up Language Detection ---
GPU detected. Using device: NVIDIA L4
Loading LangID model 'papluca/xlm-roberta-base-language-detection'...


Device set to use cuda:0


LangID Pipeline loaded successfully.
    Time taken for LangID setup: 0:00:02.316979

--- Loading Dataset ---


Generating train split: 0 examples [00:00, ? examples/s]

Error loading dataset with specific features, trying to load all: An error occurred while generating the dataset
Dataset loaded with all features.
Dataset({
    features: ['id', 'title', 'abstract', 'authors', 'createdDate', 'doi', 'arxivId', 'pubmedId', 'magId', 'oaiIds', 'links', 'publishedDate', 'updatedDate'],
    num_rows: 14265
})
Time taken for loading: 0:00:00.987081


--- Running Language Detection on Titles ---
Language detection completed in 0:00:04.534110

--- Filtering Non-English Titles ---
Found 3891 non-English titles to translate.
Time taken for filtering: 0:00:00.003045

--- Setting up Translation Model ---
Loading translation model 'facebook/m2m100_418M'...


...

Map:   0%|          | 0/3891 [00:00<?, ? examples/s]

Translation completed in 0:08:15.675633

Preparing JSONL file for download...
JSONL file created at /content/translated_titles.jsonl. Triggering download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated.

--- Translation Examples (Titles) ---
Original Lang: pt
Original Title: Complejidad descriptiva y computacional en maquinas de Turing pequenas
Translated Title: Descriptive and computing complexity in small Turing machines
--------------------
Original Lang: it
Original Title: Rotating Membranes in AdS_4xM^{1,1,1}
Translated Title: It’s a good idea for you to be able to do it.
--------------------
Original Lang: it
Original Title: Anosov branches of dynamo spectra in one dimensional plasmas
Translated Title: Anosov branches of dynamo spectrum in one-dimensional plasmas
--------------------
Original Lang: it
Original Title: Chirp Control of Sinusoidal Lattice Modes in Bose-Einstein Condensate
Translated Title: Chirp Control of Sinusoidal Milk Modes in Bose-Einstein Condensate
--------------------
Original Lang: pt
Original Title: Derived Algebraic Geometry VI: E_k Algebras
Translated Title: Derived Algebraic Geometry VI: E_k
--------------------

--- Final Language 

In [ ]:
!ls /content/drive/MyDrive/unzipped_longeval_abstracts

documents_000019.jsonl	documents_000020.jsonl	documents_000021.jsonl


In [2]:
# --- Prerequisites ---
# 1. Ensure you are using a GPU runtime in Colab (recommended for faster detection).
# 2. Ensure your .jsonl files (documents_000019.jsonl, etc.) are in the specified folder on Google Drive.
# 3. Set the folder path and the file index range in Step 3 below.

# --- Step 0: Install necessary packages ---
!pip install langdetect transformers torch accelerate datasets sentencepiece -q
print("Required packages installed.")

# --- Step 1: Import required libraries ---
from collections import Counter
import os
import io
from google.colab import drive
import datetime
import torch
import gc # Garbage Collector interface
import logging
import json # <--- Added back for manual JSON parsing

# Added for Hugging Face Transformers & Datasets
import transformers
import datasets
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset, load_dataset # Keep load_dataset in case manual parse fails fallback is added later, but not used in current logic

# --- Configure Logging ---
logging.getLogger("transformers").setLevel(logging.WARNING) # Reduce transformers verbosity

print("Libraries imported.")
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")

# --- Step 2: Mount Google Drive ---
print("\nMounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    raise

# --- Step 3: Configuration ---
# !!! 1. Set this path to the FOLDER containing the extracted .jsonl files !!!
jsonl_folder_path = '/content/drive/MyDrive/unzipped_longeval_abstracts' # <--- SET FOLDER PATH

# !!! 2. Set the START and END index for the files to process !!!
# This will process documents_000019.jsonl, documents_000020.jsonl, documents_000021.jsonl
start_file_index = 1
end_file_index = 21
filename_prefix = "documents_" # Common prefix for filenames
filename_suffix = ".jsonl"   # Common suffix for filenames

# --- Parameters ---
map_batch_size = 128             # Batch size for dataset mapping operations (language detection)
lang_id_pipeline_batch_size = 256 # Batch size for language detection pipeline itself

# --- Step 4: Language Detection Model/Pipeline Setup (Load Once) ---
print("\n--- Setting up Language Detection Model (Loading Once) ---")
start_time_setup_langid = datetime.datetime.now()
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU detected. Using device: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
    gc.collect()
else:
    device = torch.device("cpu")
    print("Warning: GPU not detected. Using CPU (might be slow).")

lang_id_model_name = "papluca/xlm-roberta-base-language-detection"
lang_id_pipeline = None
lang_id_model = None
lang_id_tokenizer = None
try:
    print(f"Loading LangID model '{lang_id_model_name}'...")
    lang_id_tokenizer = AutoTokenizer.from_pretrained(lang_id_model_name)
    lang_id_model = AutoModelForSequenceClassification.from_pretrained(lang_id_model_name).to(device)
    lang_id_pipeline = pipeline("text-classification", model=lang_id_model, tokenizer=lang_id_tokenizer, device=0 if device.type == 'cuda' else -1)
    print("LangID Pipeline loaded successfully.")
except Exception as e:
    print(f"Error loading LangID model: {e}")
    # If model loading fails, we cannot proceed
    raise

end_time_setup_langid = datetime.datetime.now()
print(f"Time taken for LangID setup: {end_time_setup_langid - start_time_setup_langid}")


# --- Step 5: Loop Through Files and Process ---

print(f"\n--- Starting Processing Loop for Files {start_file_index} to {end_file_index} ---")

for i in range(start_file_index, end_file_index + 1):
    # Construct filename with zero-padding (e.g., 000019)
    current_filename = f"{filename_prefix}{i:06d}{filename_suffix}"
    current_file_path = os.path.join(jsonl_folder_path, current_filename)

    print(f"\n===== Processing File: {current_filename} =====")

    # --- 5.1: Load Dataset (Replaced with Manual Validation) ---
    print(f"\n--- Attempting Manual JSONL Validation for: {current_filename} ---")

    valid_lines_data = [] # Store dicts like {'title': '...'}
    error_line_num = -1
    json_error_detail = None
    line_content_on_error = ""
    ds = None # Ensure ds is reset for each file loop

    try:
        # --- Check file existence before attempting to open ---
        if not os.path.exists(current_file_path):
            raise FileNotFoundError(f"File not found at path: {current_file_path}")

        # --- Explicitly open with UTF-8 ---
        print(f"   Opening file: {current_file_path} with encoding='utf-8'")
        with open(current_file_path, 'r', encoding='utf-8') as f:
            for line_idx, line in enumerate(f):
                line_num = line_idx + 1 # 1-based line number
                # Skip empty or whitespace-only lines
                stripped_line = line.strip()
                if not stripped_line:
                    continue # Skip empty lines silently
                try:
                    # Attempt to parse the line as JSON
                    json_obj = json.loads(stripped_line)

                    # --- CRITICAL CHECK: Ensure 'title' exists and is a string ---
                    if 'title' not in json_obj:
                        print(f"   !!! FATAL ERROR: Line {line_num} is missing the 'title' key. Stopping validation for {current_filename}. !!!")
                        error_line_num = line_num
                        json_error_detail = "Missing 'title' key"
                        line_content_on_error = stripped_line[:200] + "..."
                        break # Treat missing title as a fatal error for this file
                    elif json_obj['title'] is None: # Explicitly check for None
                         print(f"   !!! FATAL ERROR: 'title' on line {line_num} is null/None. Stopping validation for {current_filename}. !!!")
                         error_line_num = line_num
                         json_error_detail = "'title' is null/None"
                         line_content_on_error = stripped_line[:200] + "..."
                         break # Treat null title as a fatal error
                    elif not isinstance(json_obj['title'], str):
                         print(f"   !!! FATAL ERROR: 'title' on line {line_num} is not a string (Type: {type(json_obj['title'])}). Stopping validation for {current_filename}. !!!")
                         error_line_num = line_num
                         json_error_detail = f"'title' is not a string (Type: {type(json_obj['title'])})"
                         line_content_on_error = stripped_line[:200] + "..."
                         break # Treat wrong type as a fatal error

                    # If checks pass, store only the title (as that's all we need later)
                    valid_lines_data.append({'title': json_obj['title']})

                except json.JSONDecodeError as e:
                    # Found an invalid JSON line
                    error_line_num = line_num
                    json_error_detail = e
                    line_content_on_error = stripped_line[:200] + "..."
                    print(f"   !!! Invalid JSON detected on line {error_line_num}: {e} !!!")
                    print(f"       Content preview: {line_content_on_error}")
                    break # Stop validation on first JSON error

    except FileNotFoundError:
        print(f"!!! File Not Found: {current_file_path}. Skipping. !!!")
        # Ensure list is cleared if partially populated before error
        del valid_lines_data[:]
        gc.collect()
        continue # Skip to next file in outer loop
    except Exception as e:
        # Catch other file reading errors (e.g., permission, other encoding issues)
        print(f"!!! Error reading file {current_file_path}: {e}. Skipping. !!!")
         # Ensure list is cleared
        del valid_lines_data[:]
        gc.collect()
        continue # Skip to next file in outer loop

    # --- Check results of manual validation ---
    if error_line_num != -1:
        print(f"--- Manual validation failed for {current_filename} on line {error_line_num} ({json_error_detail}). Cannot proceed with this file. ---")
        del valid_lines_data[:] # Clean up list
        gc.collect()
        continue # Skip to next file in outer loop
    elif not valid_lines_data:
         print(f"--- No valid JSON objects with 'title' strings found in {current_filename}. Skipping. ---")
         # list is already empty or cleared
         gc.collect()
         continue # Skip to next file in outer loop
    else:
        # --- If validation succeeded, create Dataset object ---
        print(f"--- Manual validation successful for {current_filename}. Found {len(valid_lines_data)} valid lines with 'title' strings. ---")
        start_time_load = datetime.datetime.now()
        try:
            # Create dataset directly from the list of validated title dictionaries
            ds = Dataset.from_list(valid_lines_data)
            print(f"   Dataset object created successfully from validated titles for '{current_filename}'.")
             # Check title column exists (should always exist based on how we created it)
            if 'title' not in ds.column_names:
                 # This case should theoretically not happen with from_list [{'title':...}]
                 raise ValueError("'title' column unexpectedly missing after Dataset.from_list")

        except Exception as e:
            print(f"!!! Error creating Dataset object from validated titles for '{current_filename}': {e}. Skipping. !!!")
            # ds might be partially created, ensure cleanup
            if 'ds' in locals() and ds is not None: del ds
            # Also clear original list data
            del valid_lines_data[:]
            gc.collect()
            continue # Skip file if dataset creation fails

        end_time_load = datetime.datetime.now()
        # Cleanup the temporary list now data is in Dataset object
        del valid_lines_data[:]
        gc.collect()

        # Proceed if ds is created successfully
        if ds is None:
             # This case should also not happen if exception handling is correct
            print(f"!!! Dataset object is None after creation attempt for '{current_filename}'. Skipping processing. !!!")
            continue

        print(ds) # Print dataset info
        print(f"Time taken for creating Dataset object for '{current_filename}': {end_time_load - start_time_load}\n")

    # === If we reach here, 'ds' is valid and contains the 'title' column ===

    # --- 5.2: Language Detection ---
    print(f"\n--- Running Language Detection on Titles for {current_filename} ---")

    # Define the detection function (remains the same)
    def detect_language_batch(batch):
        # No need to check for 'title' existence here, as we ensured it during loading/creation
        texts = batch["title"]
        detected_lang = [None] * len(texts)
        non_empty_texts = []
        non_empty_indices = []
        for i_batch, text in enumerate(texts):
            # We already checked for None/non-string during validation, but check empty/whitespace again
            if text is None or text.strip() == "":
                detected_lang[i_batch] = "empty" # Assign specific code if needed
            else:
                non_empty_texts.append(text)
                non_empty_indices.append(i_batch)
        if non_empty_texts:
            try:
                predictions = lang_id_pipeline(
                    non_empty_texts,
                    batch_size=lang_id_pipeline_batch_size,
                    truncation=True, padding=True, max_length=512 # Truncation is important
                )
                for idx, pred in zip(non_empty_indices, predictions):
                    detected_lang[idx] = pred["label"]
            except Exception as e_pipe:
                print(f"  Error in lang_id_pipeline during batch for {current_filename}: {e_pipe}")
                for idx in non_empty_indices:
                    detected_lang[idx] = "error" # Assign 'error' if pipeline fails for a batch
        batch["detected_lang"] = detected_lang
        return batch

    start_time_map_langid = datetime.datetime.now()
    detected_ds = None # Reset variable
    try:
        # Apply mapping to the dataset 'ds' created from validated titles
        detected_ds = ds.map(detect_language_batch, batched=True, batch_size=map_batch_size, num_proc=1)
        end_time_map_langid = datetime.datetime.now()
        print(f"Language detection for '{current_filename}' completed in {end_time_map_langid - start_time_map_langid}")
    except Exception as e:
        print(f"!!! Error during language detection map for '{current_filename}': {e}. Skipping statistics for this file. !!!")
        detected_ds = None # Ensure it's None if detection fails

    # --- 5.3: Display Language Statistics ---
    print(f"\n--- Language Statistics for 'title' field in: {current_filename} ---")
    if detected_ds is not None and 'detected_lang' in detected_ds.column_names:
        try:
            language_counts = Counter(detected_ds["detected_lang"])
            total_items = len(detected_ds)
            print(f"Total titles processed in {current_filename}: {total_items}")
            print("--- Language Distribution ---")
            if total_items > 0:
                # Sort by count descending for clarity
                for lang, count in language_counts.most_common():
                    percentage = (count / total_items) * 100
                    print(f"  {lang}: {count} ({percentage:.2f}%)")
            else:
                print("  No titles found or processed.")

            # Summary Stats
            en_count = language_counts.get('en', 0)
            empty_count = language_counts.get('empty', 0)
            error_count = language_counts.get('error', 0) # Errors from pipeline only now
            non_english_count_detected = total_items - en_count - empty_count - error_count

            print("\n--- Summary ---")
            print(f"English ('en'): {en_count}")
            print(f"Non-English (excluding empty/error): {non_english_count_detected}")
            print(f"Empty/Invalid Titles ('empty'): {empty_count}")
            print(f"Pipeline Processing Errors ('error'): {error_count}")

        except Exception as e:
            print(f"!!! Error calculating statistics for '{current_filename}': {e} !!!")
    else:
        print(f"Could not calculate statistics for '{current_filename}' - detection may have failed or dataset was invalid.")

    # --- 5.4: Clean up memory for the current file ---
    print(f"--- Cleaning up memory after processing {current_filename} ---")
    # ds was created from list, detected_ds is the mapped version
    if 'ds' in locals() and ds is not None: del ds
    if 'detected_ds' in locals() and detected_ds is not None: del detected_ds
    gc.collect() # Explicitly call garbage collector

# --- End of Loop ---
print("\n===== Finished Processing All Files =====")


# --- Step 6: Final Cleanup (Model) ---
print("\n--- Final Cleanup (Language Detection Model) ---")
del lang_id_pipeline
del lang_id_model
del lang_id_tokenizer
gc.collect()
if device.type == 'cuda':
    torch.cuda.empty_cache()
print("Cleanup complete.")

print("\n--- Script Finished ---")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 46.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.1 MB/s eta 0:00:00
Required packages installed.
Libraries imported.
PyTorch version: 2.6.0+cu124
T

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

Language detection for 'documents_000019.jsonl' completed in 0:03:34.440645

--- Language Statistics for 'title' field in: documents_000019.jsonl ---
Total titles processed in documents_000019.jsonl: 100000
--- Language Distribution ---
  en: 58545 (58.55%)
  es: 7835 (7.83%)
  pt: 6800 (6.80%)
  it: 4351 (4.35%)
  fr: 4017 (4.02%)
  nl: 4002 (4.00%)
  de: 3180 (3.18%)
  ru: 2405 (2.40%)
  ur: 1819 (1.82%)
  ja: 1412 (1.41%)
  tr: 1166 (1.17%)
  pl: 1158 (1.16%)
  sw: 678 (0.68%)
  hi: 635 (0.64%)
  bg: 630 (0.63%)
  zh: 420 (0.42%)
  th: 337 (0.34%)
  el: 317 (0.32%)
  ar: 200 (0.20%)
  vi: 93 (0.09%)

--- Summary ---
English ('en'): 58545
Non-English (excluding empty/error): 41455
Empty/Invalid Titles ('empty'): 0
Pipeline Processing Errors ('error'): 0
--- Cleaning up memory after processing documents_000019.jsonl ---

===== Processing File: documents_000020.jsonl =====

--- Attempting Manual JSONL Validation for: documents_000020.jsonl ---
   Opening file: /content/drive/MyDrive/un

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Language detection for 'documents_000020.jsonl' completed in 0:03:29.716524

--- Language Statistics for 'title' field in: documents_000020.jsonl ---
Total titles processed in documents_000020.jsonl: 100000
--- Language Distribution ---
  en: 61653 (61.65%)
  es: 7413 (7.41%)
  pt: 6166 (6.17%)
  nl: 4258 (4.26%)
  it: 3995 (4.00%)
  ru: 2476 (2.48%)
  de: 2399 (2.40%)
  fr: 2340 (2.34%)
  ja: 1805 (1.80%)
  ur: 1724 (1.72%)
  pl: 1062 (1.06%)
  tr: 967 (0.97%)
  zh: 870 (0.87%)
  sw: 711 (0.71%)
  hi: 706 (0.71%)
  bg: 442 (0.44%)
  th: 395 (0.40%)
  el: 274 (0.27%)
  ar: 218 (0.22%)
  vi: 126 (0.13%)

--- Summary ---
English ('en'): 61653
Non-English (excluding empty/error): 38347
Empty/Invalid Titles ('empty'): 0
Pipeline Processing Errors ('error'): 0
--- Cleaning up memory after processing documents_000020.jsonl ---

===== Processing File: documents_000021.jsonl =====

--- Attempting Manual JSONL Validation for: documents_000021.jsonl ---
   Opening file: /content/drive/MyDrive/un

Map:   0%|          | 0/14265 [00:00<?, ? examples/s]

Language detection for 'documents_000021.jsonl' completed in 0:00:30.798329

--- Language Statistics for 'title' field in: documents_000021.jsonl ---
Total titles processed in documents_000021.jsonl: 14265
--- Language Distribution ---
  en: 10374 (72.72%)
  nl: 677 (4.75%)
  pt: 672 (4.71%)
  it: 553 (3.88%)
  es: 515 (3.61%)
  fr: 304 (2.13%)
  ru: 233 (1.63%)
  ur: 139 (0.97%)
  hi: 125 (0.88%)
  de: 124 (0.87%)
  sw: 119 (0.83%)
  ja: 110 (0.77%)
  tr: 101 (0.71%)
  pl: 57 (0.40%)
  th: 45 (0.32%)
  zh: 41 (0.29%)
  el: 28 (0.20%)
  bg: 26 (0.18%)
  ar: 17 (0.12%)
  vi: 5 (0.04%)

--- Summary ---
English ('en'): 10374
Non-English (excluding empty/error): 3891
Empty/Invalid Titles ('empty'): 0
Pipeline Processing Errors ('error'): 0
--- Cleaning up memory after processing documents_000021.jsonl ---

===== Finished Processing All Files =====

--- Final Cleanup (Language Detection Model) ---
Cleanup complete.

--- Script Finished ---
